# Instalando bibliotecas

In [80]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import openpyxl
import time


# Inicializando navegador


In [81]:
options = Options()
# options.add_argument("--headless")  # Executa sem abrir janela
options.add_argument("--no-sandbox")

# Aponta para o chromedriver na raiz do projeto Windows
service = Service(executable_path="./chromedriver-linux64/chromedriver.exe")
# Aponta para o chromedriver na raiz do projeto Linux
#service = Service(executable_path="./chromedriver-linux64/chromedriver")
driver = webdriver.Chrome(service=service, options=options)

# Acessa a página dos campeões
driver.get('https://www.op.gg/champions')

# Scrollando a página


# Definindo as listas do scraping



In [82]:
# Lista de regiões
regions = [
    "global", "me", "sea", "tw", "vn", "br", "euw", "eune", "jp", "kr", 
    "lan", "las", "na", "oce", "tr", "ru"
]

# Lista de lanes
lanes = ["All", "Top", "Jungle", "Middle", "Bottom", "Support"]

# URL base
base_url = "https://op.gg/lol/champions?region={region}&type=ranked"


all_data = [] 



# Fazendo a raspagem


In [83]:
# Armazenamento de todos os dados
all_data = []

# Loop por região
for region in regions:
    url = base_url.format(region=region)
    print(f"\n==== Acessando região: {region.upper()} ====\n")
    driver.get(url)
    time.sleep(3)  # Aguarda o carregamento inicial

    # Loop por lane
    for lane in lanes:
        print(f"\n-- Coletando dados da lane: {lane} --")

        # Clicar na aba da lane
        spans = driver.find_elements(By.TAG_NAME, "span")
        for span in spans:
            if span.text.strip().lower() == lane.lower():
                try:
                    span.click()
                    time.sleep(2)
                    break
                except:
                    print(f"Erro ao clicar na lane {lane}")
        
        # Scroll infinito até o fim da página
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Coletar tabela
        try:
            tbody = driver.find_element(By.TAG_NAME, 'tbody')
            rows = tbody.find_elements(By.TAG_NAME, 'tr')
            for row in rows:
                cells = row.find_elements(By.TAG_NAME, 'td')
                row_data = [cell.text for cell in cells]
                if row_data:
                    full_row = [region, lane] + row_data
                    all_data.append(full_row)

                    # Print no console do dado raspado
                    print(full_row)
        except Exception as e:
            print(f"Erro ao processar dados da lane {lane} na região {region}: {e}")

# Fechar navegador
driver.quit()

# Ajuste de colunas conforme o número de colunas encontradas (ajuste se necessário)
expected_columns = max(len(row) for row in all_data)
columns = ['Region', 'Lane'] + [f'Col{i}' for i in range(1, expected_columns - 1)]

# Filtra linhas inconsistentes
all_data = [row for row in all_data if len(row) == len(columns)]

# Criar DataFrame e exportar para Excel
df = pd.DataFrame(all_data, columns=columns)
df.to_excel("todos_champions.xlsx", index=False)

print("\n✅ Exportação concluída: 'todos_champions.xlsx'")


==== Acessando região: GLOBAL ====


-- Coletando dados da lane: All --
['global', 'All', '1', 'Kayle', '', '', '53.11%', '3.25%', '', '']
['global', 'All', '2', "Cho'Gath", '', '', '53.07%', '3.35%', '', '']
['global', 'All', '3', 'Warwick', '', '', '52.83%', '3.35%', '', '']
['global', 'All', '4', 'Braum', '', '', '52.64%', '8.14%', '', '']
['global', 'All', '5', 'Sett', '', '', '52.61%', '7.4%', '', '']
['global', 'All', '6', 'Jinx', '', '', '52.57%', '13.04%', '', '']
['global', 'All', '7', 'Galio', '', '', '52.5%', '4.79%', '', '']
['global', 'All', '8', 'Volibear', '', '', '52.39%', '3.8%', '', '']
['global', 'All', '9', 'Nami', '', '', '52.35%', '14.03%', '', '']
['global', 'All', '10', 'Milio', '', '', '52.27%', '7.17%', '', '']
['global', 'All', '11', 'Malphite', '', '', '52.26%', '4.53%', '', '']
['global', 'All', '12', 'Annie', '', '', '52.18%', '5.1%', '', '']
['global', 'All', '13', 'Elise', '', '', '52.16%', '4.45%', '', '']
['global', 'All', '14', 'Shen', '', '', '52.13

In [86]:
print(df[10])

KeyError: 10

In [84]:
data[19]

NameError: name 'data' is not defined